In [1]:
import numpy as np 
import pandas as pd 
import os

## Stage 1: Analyze men's match data and create a prediction model for men's teams

### Calculating the winning rate for each team in the men's conference Tourney games:

In [2]:
# Load conference tournament match data
conference_games = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MConferenceTourneyGames.csv')

# Convert match results into a format where each team has its own record
# Create a DataFrame for winning teams with a win label
conference_games_wins = conference_games[['WTeamID']].copy()
conference_games_wins['Win'] = 1
conference_games_wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# Create a DataFrame for losing teams with a loss label
conference_games_losses = conference_games[['LTeamID']].copy()
conference_games_losses['Win'] = 0
conference_games_losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Combine both DataFrames to create a unified match record per team
conference_team_stats = pd.concat([conference_games_wins, conference_games_losses], ignore_index=True)

# Calculate total matches played and win rate for each team in conference tournaments
conference_summary = (
    conference_team_stats.groupby('TeamID')
    .agg(TotalGames=('Win', 'count'), ConferenceWins=('Win', 'sum'))
    .reset_index()
)

# Compute win rate in conference tournaments
conference_summary['TeamConferenceWinRate'] = (
    conference_summary['ConferenceWins'] / conference_summary['TotalGames']
)

# Add an indicator feature to specify that this data is from a conference tournament
conference_summary['IsConferenceTourney'] = 1

# Save the processed conference tournament feature data
conference_summary.to_csv('/kaggle/working/conference_summary.csv', index=False)

# Display a preview of the processed data
print("Conference tournament summary preview:")
print(conference_summary.head())


Conference tournament summary preview:
   TeamID  TotalGames  ConferenceWins  TeamConferenceWinRate  \
0    1101          10               7               0.700000   
1    1102          30               6               0.200000   
2    1103          56              38               0.678571   
3    1104          44              23               0.522727   
4    1105          28              12               0.428571   

   IsConferenceTourney  
0                    1  
1                    1  
2                    1  
3                    1  
4                    1  


### Calculate the winning rate for each team and the average point difference in the tournament under pressure for men:

In [3]:
# Load tournament match data
tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')

# Calculate tournament wins and losses for each team
tourney_wins = tourney_results.groupby('WTeamID').size().reset_index(name='TournamentWins')
tourney_losses = tourney_results.groupby('LTeamID').size().reset_index(name='TournamentLosses')

# Rename columns for consistency before merging
tourney_wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)
tourney_losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Merge win and loss data for each team
tourney_stats = pd.merge(tourney_wins, tourney_losses, on='TeamID', how='outer')

# Replace missing values with zero and ensure integer data type
tourney_stats.fillna(0, inplace=True)
tourney_stats = tourney_stats.astype({'TournamentWins': 'int', 'TournamentLosses': 'int'})

# Calculate tournament win rate while handling cases where no games were played
tourney_stats['TournamentWinRate'] = tourney_stats.apply(
    lambda row: row['TournamentWins'] / (row['TournamentWins'] + row['TournamentLosses']) 
    if (row['TournamentWins'] + row['TournamentLosses']) > 0 else 0, axis=1
)

# Calculate average score difference in tournament games
tourney_results['ScoreDiff'] = tourney_results['WScore'] - tourney_results['LScore']
score_diff = tourney_results.groupby('WTeamID')['ScoreDiff'].mean().reset_index()
score_diff.rename(columns={'WTeamID': 'TeamID', 'ScoreDiff': 'AvgTournamentScoreDiff'}, inplace=True)

# Merge calculated features
tourney_features = pd.merge(
    tourney_stats[['TeamID', 'TournamentWinRate']], 
    score_diff, 
    on='TeamID', 
    how='left'
)

# Replace missing values in score difference with zero
tourney_features['AvgTournamentScoreDiff'] = tourney_features['AvgTournamentScoreDiff'].fillna(0)

# Save the processed tournament features
tourney_features.to_csv('/kaggle/working/tourney_features.csv', index=False)

# Display a preview of the results
print("Tournament features preview:")
print(tourney_features[['TeamID', 'TournamentWinRate', 'AvgTournamentScoreDiff']].head())


Tournament features preview:
   TeamID  TournamentWinRate  AvgTournamentScoreDiff
0    1101           0.333333                1.000000
1    1102           0.000000                0.000000
2    1103           0.000000                0.000000
3    1104           0.574468               10.555556
4    1105           0.000000                0.000000


### Calculating the percentage of successful shots for each team and the rate of extra time from the detailed results of the men's tournaments:

In [4]:
# Load detailed tournament results data
tourney_detailed = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyDetailedResults.csv')

# Extract relevant columns for winning teams: TeamID, Field Goals Made (FGM), Field Goals Attempted (FGA), and Overtime count
winners = tourney_detailed[['WTeamID', 'WFGM', 'WFGA', 'NumOT']].copy()
winners.rename(columns={'WTeamID': 'TeamID', 'WFGM': 'FGM', 'WFGA': 'FGA'}, inplace=True)

# Extract relevant columns for losing teams: TeamID, Field Goals Made (FGM), Field Goals Attempted (FGA), and Overtime count
losers = tourney_detailed[['LTeamID', 'LFGM', 'LFGA', 'NumOT']].copy()
losers.rename(columns={'LTeamID': 'TeamID', 'LFGM': 'FGM', 'LFGA': 'FGA'}, inplace=True)

# Combine winners and losers into a single dataset for shot statistics
teams_shots = pd.concat([winners, losers])

# Calculate the field goal percentage for each team
teams_shots['FG%'] = np.where(teams_shots['FGA'] > 0, teams_shots['FGM'] / teams_shots['FGA'], 0)

# Compute the average field goal percentage for each team across all tournament games
fg_percentage = teams_shots.groupby('TeamID')['FG%'].mean().reset_index()
fg_percentage.rename(columns={'FG%': 'AvgFieldGoalPercentage'}, inplace=True)

# Compute the average number of overtime games per team
ot_games = teams_shots.groupby('TeamID')['NumOT'].mean().reset_index()
ot_games.rename(columns={'NumOT': 'AvgOTGames'}, inplace=True)

# Merge the computed features into a single dataset
detailed_features = fg_percentage.merge(ot_games, on='TeamID', how='left')

# Save the final dataset
detailed_features.to_csv('/kaggle/working/detailed_features.csv', index=False)

# Display the first few rows of the final dataset
print("Detailed tournament features preview:")
print(detailed_features.head())


Detailed tournament features preview:
   TeamID  AvgFieldGoalPercentage  AvgOTGames
0    1101                0.305836    0.000000
1    1102                0.474046    0.000000
2    1103                0.370330    0.000000
3    1104                0.458859    0.043478
4    1105                0.389831    0.000000


### Calculating Win Rate and Average Points Difference in Regular Season Under Pressure for Men:

In [5]:
# Load Regular Season Results Data
season_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')

# Compute the number of wins for each team
season_wins = season_results.groupby('WTeamID').size().reset_index(name='SeasonWins')
season_wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# Compute the number of losses for each team
season_losses = season_results.groupby('LTeamID').size().reset_index(name='SeasonLosses')
season_losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Merge wins and losses into a single dataset (ensuring all teams are included)
season_stats = pd.merge(season_wins, season_losses, on='TeamID', how='outer').fillna(0)

# Ensure numerical data types
season_stats[['SeasonWins', 'SeasonLosses']] = season_stats[['SeasonWins', 'SeasonLosses']].astype(int)

# Compute the win rate for each team
season_stats['SeasonWinRate'] = season_stats['SeasonWins'] / (season_stats['SeasonWins'] + season_stats['SeasonLosses'])

# Calculate the average points difference in the regular season
season_results['ScoreDiff'] = season_results['WScore'] - season_results['LScore']
avg_score_diff = season_results.groupby('WTeamID')['ScoreDiff'].mean().reset_index()
avg_score_diff.rename(columns={'WTeamID': 'TeamID', 'ScoreDiff': 'AvgSeasonScoreDiff'}, inplace=True)

# Merge the computed features into a single dataset
season_features = season_stats.merge(avg_score_diff, on='TeamID', how='left')

# Fill missing values for score difference with 0
season_features['AvgSeasonScoreDiff'] = season_features['AvgSeasonScoreDiff'].fillna(0)

# Save the final dataset
season_features.to_csv('/kaggle/working/season_features.csv', index=False)

# Display a preview of the dataset
print("Regular Season Features Preview:")
print(season_features.head())


Regular Season Features Preview:
   TeamID  SeasonWins  SeasonLosses  SeasonWinRate  AvgSeasonScoreDiff
0    1101         148           167       0.469841           11.243243
1    1102         395           710       0.357466           11.903797
2    1103         689           484       0.587383           12.503628
3    1104         795           464       0.631454           13.695597
4    1105         244           456       0.348571           10.188525


### Calculating successful shooting percentage and average points conceded for each team from the detailed results of the men's regular season

In [6]:
# Load detailed results for the regular season
season_detailed = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv')

# Extract relevant stats for the winning team (field goals made/attempted, points scored, and points allowed)
winners = season_detailed[['WTeamID', 'WFGM', 'WFGA', 'LScore', 'WScore']].copy()
winners.rename(columns={'WTeamID': 'TeamID', 'WFGM': 'FGM', 'WFGA': 'FGA', 
                          'LScore': 'PointsAllowed', 'WScore': 'PointsScored'}, inplace=True)

# Extract relevant stats for the losing team (same stats as winners)
losers = season_detailed[['LTeamID', 'LScore', 'WFGM', 'WFGA', 'WScore']].copy()
losers.rename(columns={'LTeamID': 'TeamID', 'LScore': 'PointsScored', 'WFGM': 'FGM', 
                         'WFGA': 'FGA', 'WScore': 'PointsAllowed'}, inplace=True)

# Combine winners and losers into a single dataset
teams_stats = pd.concat([winners, losers], ignore_index=True)

# Ensure FGA values are non-negative
teams_stats['FGA'] = teams_stats['FGA'].clip(lower=0)

# Calculate field goal percentage (avoiding division by zero)
teams_stats['FG%'] = np.where(teams_stats['FGA'] > 0, teams_stats['FGM'] / teams_stats['FGA'], 0)

# Compute the average field goal percentage for each team
fg_percentage_season = teams_stats.groupby('TeamID')['FG%'].mean().reset_index().rename(
    columns={'FG%': 'AvgFieldGoalPercentageSeason'}
)

# Compute the average points allowed (defensive strength) for each team
defensive_strength = teams_stats.groupby('TeamID')['PointsAllowed'].mean().reset_index().rename(
    columns={'PointsAllowed': 'AvgPointsAllowed'}
)

# Compute the average points scored (offensive strength) for each team
offensive_strength= teams_stats.groupby('TeamID')['PointsScored'].mean().reset_index().rename(
    columns={'PointsScored': 'AvgPointsScored'}
)

# Merge all extracted features into a single dataset
season_detailed_features = fg_percentage_season.merge(defensive_strength, on='TeamID', how='left')
season_detailed_features = season_detailed_features.merge(offensive_strength, on='TeamID', how='left')

# Fill NaN values with 0
season_detailed_features = season_detailed_features.fillna(0)

# Save the feature dataset
season_detailed_features.to_csv('/kaggle/working/season_detailed_features.csv', index=False)

# Display the first few rows of the final dataset
print(season_detailed_features.head())

   TeamID  AvgFieldGoalPercentageSeason  AvgPointsAllowed  AvgPointsScored
0    1101                      0.487839         71.285714        69.761905
1    1102                      0.490937         65.689759        63.759036
2    1103                      0.470686         67.280851        72.295035
3    1104                      0.465272         68.543296        73.317039
4    1105                      0.458149         71.444623        64.778491


### Calculating win rate and performance difference for each team in a men's secondary game under pressure:

In [7]:
# Load the secondary tournament results
secondary_tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyCompactResults.csv')

# Calculate the number of wins and losses in the secondary tournament
secondary_tourney_wins = secondary_tourney_results.groupby('WTeamID').size().reset_index(name='SecondaryTourneyWins')
secondary_tourney_losses = secondary_tourney_results.groupby('LTeamID').size().reset_index(name='SecondaryTourneyLosses')

# Merge win and loss data
secondary_tourney_stats = pd.merge(
    secondary_tourney_wins, secondary_tourney_losses, 
    left_on='WTeamID', right_on='LTeamID', how='outer'
).fillna(0)

# Assign TeamID correctly
secondary_tourney_stats['TeamID'] = secondary_tourney_stats['WTeamID'].fillna(secondary_tourney_stats['LTeamID']).astype(int)

# Compute win rate in the secondary tournament, handling division by zero
secondary_tourney_stats['SecondaryTourneyWinRate'] = np.where(
    (secondary_tourney_stats['SecondaryTourneyWins'] + secondary_tourney_stats['SecondaryTourneyLosses']) > 0,
    secondary_tourney_stats['SecondaryTourneyWins'] / (secondary_tourney_stats['SecondaryTourneyWins'] + secondary_tourney_stats['SecondaryTourneyLosses']),
    0
)

# Define games played under pressure:
# Conditions: overtime (NumOT > 0) or close games (point difference <= 5)
secondary_tourney_results['ScoreDiff'] = secondary_tourney_results['WScore'] - secondary_tourney_results['LScore']
pressure_games = secondary_tourney_results[
    (secondary_tourney_results['NumOT'] > 0) | (secondary_tourney_results['ScoreDiff'].abs() <= 5)
]

# Compute win and loss counts under pressure
pressure_wins = pressure_games.groupby('WTeamID').size().reset_index(name='PressureWins')
pressure_losses = pressure_games.groupby('LTeamID').size().reset_index(name='PressureLosses')

# Merge pressure-based performance data
pressure_stats = pd.merge(
    pressure_wins, pressure_losses, 
    left_on='WTeamID', right_on='LTeamID', how='outer'
).fillna(0)

# Assign TeamID correctly
pressure_stats['TeamID'] = pressure_stats['WTeamID'].fillna(pressure_stats['LTeamID']).astype(int)

# Compute win rate under pressure, handling division by zero
pressure_stats['SecondaryTourneyPressureWinRate'] = np.where(
    (pressure_stats['PressureWins'] + pressure_stats['PressureLosses']) > 0,
    pressure_stats['PressureWins'] / (pressure_stats['PressureWins'] + pressure_stats['PressureLosses']),
    0
)

# Merge extracted features
secondary_tourney_features = pd.merge(
    secondary_tourney_stats[['TeamID', 'SecondaryTourneyWinRate']],
    pressure_stats[['TeamID', 'SecondaryTourneyPressureWinRate']],
    on='TeamID',
    how='left'
).fillna(0)

# Save the feature file
secondary_tourney_features.to_csv('/kaggle/working/secondary_tourney_features.csv', index=False)

# Preview results
print("Secondary Tournament Features Preview:")
print(secondary_tourney_features.head())


Secondary Tournament Features Preview:
   TeamID  SecondaryTourneyWinRate  SecondaryTourneyPressureWinRate
0    1101                 0.500000                         0.500000
1    1102                 0.625000                         0.666667
2    1103                 0.250000                         0.428571
3    1104                 0.538462                         0.375000
4    1105                 0.500000                         0.000000


### Calculating each team's experience in the first division using MTeams data:

In [8]:
# Load men's team data
teams = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MTeams.csv')

# Check if required columns exist
required_columns = {'TeamID', 'FirstD1Season', 'LastD1Season'}
if not required_columns.issubset(teams.columns):
    missing_columns = required_columns - set(teams.columns)
    raise KeyError(f"The MTeams file is missing the following columns: {missing_columns}")

# Calculate each team's experience in the first division (D1)
teams['D1Experience'] = teams['LastD1Season'] - teams['FirstD1Season'] + 1

# Save extracted feature file
teams.to_csv('/kaggle/working/teams.csv', index=False)

# Preview final results (check if 'TeamName' exists before using it)
preview_columns = ['TeamID', 'D1Experience']
if 'TeamName' in teams.columns:
    preview_columns.insert(1, 'TeamName')

print("Teams Data Preview:")
print(teams[preview_columns].head())


Teams Data Preview:
   TeamID     TeamName  D1Experience
0    1101  Abilene Chr            12
1    1102    Air Force            41
2    1103        Akron            41
3    1104      Alabama            41
4    1105  Alabama A&M            26


### Merge all extracted feature files for men:

In [9]:
# Load all feature files
conference_summary = pd.read_csv('conference_summary.csv', low_memory=False)
tourney_features = pd.read_csv('tourney_features.csv', low_memory=False)
detailed_features = pd.read_csv('detailed_features.csv', low_memory=False)
season_features = pd.read_csv('season_features.csv', low_memory=False)
season_detailed_features = pd.read_csv('season_detailed_features.csv', low_memory=False)
secondary_tourney_features = pd.read_csv('secondary_tourney_features.csv', low_memory=False)
teams = pd.read_csv('teams.csv', low_memory=False)

# Merge all files based on TeamID
merged_df = (
    conference_summary[['TeamID', 'TeamConferenceWinRate']]
    .merge(tourney_features[['TeamID', 'TournamentWinRate', 'AvgTournamentScoreDiff']], on='TeamID', how='left')
    .merge(detailed_features[['TeamID', 'AvgFieldGoalPercentage', 'AvgOTGames']], on='TeamID', how='left')
    .merge(season_features[['TeamID', 'SeasonWinRate', 'AvgSeasonScoreDiff']], on='TeamID', how='left')
    .merge(season_detailed_features[['TeamID', 'AvgFieldGoalPercentageSeason', 'AvgPointsAllowed', 'AvgPointsScored']], on='TeamID', how='left')  # ✅ إضافة AvgPointsScored
    .merge(secondary_tourney_features[['TeamID', 'SecondaryTourneyWinRate', 'SecondaryTourneyPressureWinRate']], on='TeamID', how='left')
    .merge(teams[['TeamID', 'D1Experience']], on='TeamID', how='left')
)

# Fill missing values with 0 (important to avoid NaN issues)
merged_df.fillna(0, inplace=True)

# Save the merged file
merged_df.to_csv('/kaggle/working/merged_team_features.csv', index=False)

# Show preliminary results
print("Merged Data Preview:")
print(merged_df.head())

# Check for missing values
print("\nData Overview:")
print(merged_df.info())


Merged Data Preview:
   TeamID  TeamConferenceWinRate  TournamentWinRate  AvgTournamentScoreDiff  \
0    1101               0.700000           0.333333                1.000000   
1    1102               0.200000           0.000000                0.000000   
2    1103               0.678571           0.000000                0.000000   
3    1104               0.522727           0.574468               10.555556   
4    1105               0.428571           0.000000                0.000000   

   AvgFieldGoalPercentage  AvgOTGames  SeasonWinRate  AvgSeasonScoreDiff  \
0                0.305836    0.000000       0.469841           11.243243   
1                0.474046    0.000000       0.357466           11.903797   
2                0.370330    0.000000       0.587383           12.503628   
3                0.458859    0.043478       0.631454           13.695597   
4                0.389831    0.000000       0.348571           10.188525   

   AvgFieldGoalPercentageSeason  AvgPointsAllow

### Create all possible combinations of teams so that each pair consists of two different men's teams:

In [10]:
from itertools import combinations

# Load the dataset containing team features
file_path = "merged_team_features.csv"
df = pd.read_csv(file_path)

# Generate all possible team matchups (unique pairs of teams)
team_pairs = list(combinations(df['TeamID'], 2))

# Define column names for the new DataFrame
columns = ['TeamID1', 'TeamID2'] + [f'{col}_diff' for col in df.columns if col != 'TeamID'] + ['WinProbability']

# Initialize a list to store the processed matchups
data = []

# Iterate through all possible team pairs
for team1, team2 in team_pairs:
    # Retrieve data for both teams
    team1_data = df[df['TeamID'] == team1].squeeze()
    team2_data = df[df['TeamID'] == team2].squeeze()
    
    # Skip if data for any team is missing
    if team1_data.empty or team2_data.empty:
        continue  
    
    # Store team IDs
    row = {'TeamID1': team1, 'TeamID2': team2}
    
    # Compute feature differences between the two teams
    feature_diffs = []
    for col in df.columns:
        if col != 'TeamID':
            diff = team1_data[col] - team2_data[col]
            if pd.isna(diff):  # Handle missing values
                diff = 0
            row[f'{col}_diff'] = diff
            feature_diffs.append(diff)
    
    # Compute the estimated win probability using the logistic function
    feature_sum = sum(feature_diffs)
    
    if np.isfinite(feature_sum):  # Ensure the value is valid before applying the function
        row['WinProbability'] = round(1 / (1 + np.exp(-feature_sum)), 2)
    else:
        row['WinProbability'] = 0.5  # Assign a neutral probability in case of invalid values
    
    # Append the computed row to the data list
    data.append(row)

# Convert the list into a DataFrame
pairs_df = pd.DataFrame(data, columns=columns)

# Save the results to a CSV file
pairs_df.to_csv("/kaggle/working/team_pairs_diff.csv", index=False)

# Print success message
print(" The file 'team_pairs_diff.csv' was created successfully!")


 The file 'team_pairs_diff.csv' was created successfully!


In [11]:
team_pairs_diff = pd.read_csv("/kaggle/working/team_pairs_diff.csv")
team_pairs_diff

,TeamID1,TeamID2,TeamConferenceWinRate_diff,TournamentWinRate_diff,AvgTournamentScoreDiff_diff,AvgFieldGoalPercentage_diff,AvgOTGames_diff,SeasonWinRate_diff,AvgSeasonScoreDiff_diff,AvgFieldGoalPercentageSeason_diff,AvgPointsAllowed_diff,AvgPointsScored_diff,SecondaryTourneyWinRate_diff,SecondaryTourneyPressureWinRate_diff,D1Experience_diff,WinProbability
0,1101,1102,0.500000,0.333333,1.000000,-0.168210,0.000000,0.112375,-0.660554,-0.003098,5.595955,6.002869,-0.125000,-0.166667,-29.0,0.00
1,1101,1103,0.021429,0.333333,1.000000,-0.064494,0.000000,-0.117542,-1.260385,0.017153,4.004863,-2.533131,0.250000,0.071429,-29.0,0.00
2,1101,1104,0.177273,-0.241135,-9.555556,-0.153023,-0.043478,-0.161612,-2.452354,0.022567,2.742418,-3.555134,-0.038462,0.125000,-29.0,0.00
3,1101,1105,0.271429,0.333333,1.000000,-0.083995,0.000000,0.121270,1.054719,0.029690,-0.158909,4.983414,0.000000,0.500000,-14.0,0.00
4,1101,1106,0.159459,0.333333,1.000000,-0.044547,0.000000,0.047839,1.322303,0.028813,2.096607,4.600029,0.500000,0.500000,-29.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65698,1474,1477,0.166667,0.000000,0.000000,0.000000,0.000000,0.201860,3.021905,0.001367,2.927547,8.930389,0.000000,0.000000,0.0,1.00
65699,1474,1478,0.000000,0.000000,0.000000,0.000000,0.000000,0.130907,-3.369674,0.001159,2.294381,5.494891,0.000000,0.000000,1.0,1.00
65700,1475,1477,-0.333333,0.000000,0.000000,0.000000,0.000000,0.023979,2.200000,-0.020358,0.480954,3.515895,0.000000,0.000000,0.0,1.00
65701,1475,1478,-0.500000,0.000000,0.000000,0.000000,0.000000,-0.046974,-4.191579,-0.020567,-0.152213,0.080397,0.000000,0.000000,1.0,0.02


### Model for predicting the probability of the team with the smallest ID for men winning:

### Using XGBoost:

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import optuna

# Define feature columns (all '_diff' columns) and target variable ('WinProbability')
feature_columns = [col for col in pairs_df.columns if col.endswith('_diff')]
X = pairs_df[feature_columns]
y = pairs_df['WinProbability']

# Split the dataset into training (80%) and testing (20%) ensuring reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values using SimpleImputer (replace NaNs with the median)
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

print(f"Training set size: {X_train.shape}, Testing set size: {X_test.shape}")

# Define the Optuna optimization function
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),  # L2 regularization
        'random_state': 42
    }
    
    # Train the model with early stopping to prevent overfitting
    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
    
    # Make predictions and clip values between 0 and 1
    y_pred = np.clip(model.predict(X_test), 0, 1)
    
    # Return Mean Squared Error (MSE) for evaluation
    return mean_squared_error(y_test, y_pred)

# Run Optuna to optimize hyperparameters
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=75)  # Increased trials for better optimization

# Retrieve the best hyperparameters found by Optuna
best_params = study.best_params
print("Best Hyperparameters: ", best_params)

# Train the final model with optimized hyperparameters and early stopping
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)

# Generate final predictions and ensure values are between 0 and 1
y_pred_full = np.clip(best_model.predict(X), 0, 1)
pairs_df['WinProbability'] = y_pred_full

# Format output to match the required submission format
season = 2025  # Adjust the season as needed
pairs_df['ID'] = pairs_df.apply(lambda row: f"{season}_{int(row['TeamID1'])}_{int(row['TeamID2'])}", axis=1)
pairs_df['Pred'] = pairs_df['WinProbability'].round(1)  # Round probabilities to one decimal place
submission_df = pairs_df[['ID', 'Pred']]

# Save the submission file
submission_df.to_csv("/kaggle/working/submission_m.csv", index=False)
print("The file 'submission_m.csv' was created successfully!")

# Calculate and display MSE on both training and test sets
y_pred_train = np.clip(best_model.predict(X_train), 0, 1)
y_pred_test = np.clip(best_model.predict(X_test), 0, 1)
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
print(f"Mean Squared Error on Training Set: {train_mse:.4f}")
print(f"Mean Squared Error on Test Set: {test_mse:.4f}")

[I 2025-03-03 08:41:17,467] A new study created in memory with name: no-name-0b4e7cd8-fa95-45ac-81aa-0ba9072ff8c1


Training set size: (52562, 13), Testing set size: (13141, 13)


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2025-03-03 08:41:19,821] Trial 0 finished with value: 0.0061707521417067685 and parameters: {'n_estimators': 287, 'learning_rate': 0.19063571821788408, 'max_depth': 7, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 1.5599452033620265, 'reg_lambda': 1.5227525095137953}. Best is trial 0 with value: 0.0061707521417067685.
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2025-03-03 08:41:23,302] Trial 1 finished with value: 0.006595696811029697 and parameters: {'n_estimators': 533, 'le

Best Hyperparameters:  {'n_estimators': 563, 'learning_rate': 0.13178241818551803, 'max_depth': 8, 'subsample': 0.7749824646341003, 'colsample_bytree': 0.9729415941858711, 'reg_alpha': 0.3136068892181022, 'reg_lambda': 3.165743804154165}
The file 'submission_m.csv' was created successfully!
Mean Squared Error on Training Set: 0.0004
Mean Squared Error on Test Set: 0.0041


In [13]:
submission_m = pd.read_csv("/kaggle/working/submission_m.csv") 
submission_m

,ID,Pred
0,2025_1101_1102,0.0
1,2025_1101_1103,0.0
2,2025_1101_1104,0.0
3,2025_1101_1105,0.1
4,2025_1101_1106,0.0
...,...,...
65698,2025_1474_1477,1.0
65699,2025_1474_1478,1.0
65700,2025_1475_1477,1.0
65701,2025_1475_1478,0.1


### Ensure that there are no empty values ​​in the prediction model:

In [14]:
# Download the file
submission_m = pd.read_csv('/kaggle/working/submission_m.csv')

# Check for blank values in ID column
missing_id = submission_m['ID'].isna().sum()
print(f"Number of empty values in ID: {missing_id}")

# Extract TeamID1 and TeamID2 from ID
submission_m[['Season', 'TeamID1', 'TeamID2']] = submission_m['ID'].str.split('_', expand=True)

# Check for empty values in TeamID1 and TeamID2
missing_teamid1 = submission_m['TeamID1'].isna().sum()
missing_teamid2 = submission_m['TeamID2'].isna().sum()

print(f"Number of empty values ​​in TeamID1: {missing_teamid1}")
print(f"Number of empty values ​​in TeamID2: {missing_teamid2}")

# If there are no empty values
if missing_id == 0 and missing_teamid1 == 0 and missing_teamid2 == 0:
    print("\nThere are no empty values in ID, TeamID1, or TeamID2.")
else:
    print("\nThere are blank values in some columns, check the data.")

Number of empty values in ID: 0
Number of empty values ​​in TeamID1: 0
Number of empty values ​​in TeamID2: 0

There are no empty values in ID, TeamID1, or TeamID2.


#### 🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆

## Stage 2: Analyze women’s match data and create a prediction model for women’s teams

### Calculating the win rate for each team in women's conference tournament games:

In [15]:
# Load conference tournament game data
conference_games_w = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WConferenceTourneyGames.csv')

# Extract winning teams and mark them with "Win = 1"
conference_games_wins_w = conference_games_w[['WTeamID']].copy()
conference_games_wins_w['Win'] = 1
conference_games_wins_w.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# Extract losing teams and mark them with "Win = 0"
conference_games_losses_w = conference_games_w[['LTeamID']].copy()
conference_games_losses_w['Win'] = 0
conference_games_losses_w.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Combine winning and losing teams into one dataset, ensuring each team has its match record
conference_team_stats_w = pd.concat([conference_games_wins_w, conference_games_losses_w])

# Compute total games played and number of wins per team
conference_summary_w = (
    conference_team_stats_w.groupby('TeamID')
    .agg(TotalGames=('Win', 'count'), ConferenceWins=('Win', 'sum'))
    .reset_index()
)

# Ensure there are no division errors by replacing NaN values with 0
conference_summary_w['TotalGames'] = conference_summary_w['TotalGames'].fillna(0)
conference_summary_w['ConferenceWins'] = conference_summary_w['ConferenceWins'].fillna(0)

# Compute win rate while avoiding division by zero
conference_summary_w['TeamConferenceWinRate'] = (
    conference_summary_w['ConferenceWins'] / conference_summary_w['TotalGames']
).fillna(0)  # In case any division by zero occurs, replace NaN with 0

# Add a feature that specifies that this data is for a conference tournament
conference_summary_w['IsConferenceTourney'] = 1

# Save the extracted conference statistics
conference_summary_w.to_csv('/kaggle/working/conference_summary_w.csv', index=False)

# Show preliminary results
conference_summary_w.head()

,TeamID,TotalGames,ConferenceWins,TeamConferenceWinRate,IsConferenceTourney
0,3101,10,5,0.500000,1
1,3102,29,6,0.206897,1
2,3103,30,11,0.366667,1
3,3104,31,8,0.258065,1
4,3105,24,4,0.166667,1


### Calculate the win rate for each team and the average point difference in the tournament under pressure for women:

In [16]:
# Download tournament match data under pressure
tourney_results_w = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')

# Calculate the tournament win rate for each team:
# 1- Calculate the number of wins for each winning team.
tourney_wins_w = tourney_results_w.groupby('WTeamID').size().reset_index(name='TournamentWins')
tourney_wins_w.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# 2- Calculate the number of losses for each losing team.
tourney_losses_w = tourney_results_w.groupby('LTeamID').size().reset_index(name='TournamentLosses')
tourney_losses_w.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 3- Merge win and loss data and ensure no NaN values.
tourney_stats_w = pd.merge(tourney_wins_w, tourney_losses_w, on='TeamID', how='outer').fillna(0)

# 4- Ensure correct data types
tourney_stats_w['TournamentWins'] = tourney_stats_w['TournamentWins'].astype(int)
tourney_stats_w['TournamentLosses'] = tourney_stats_w['TournamentLosses'].astype(int)

# 5- Calculating the winning rate in the tournament.
tourney_stats_w['TournamentWinRate'] = tourney_stats_w['TournamentWins'] / (
    tourney_stats_w['TournamentWins'] + tourney_stats_w['TournamentLosses']
)

# Calculate the average points difference in the tournament
tourney_results_w['ScoreDiff'] = tourney_results_w['WScore'] - tourney_results_w['LScore']
score_diff_w = tourney_results_w.groupby('WTeamID')['ScoreDiff'].mean().reset_index()
score_diff_w.rename(columns={'WTeamID': 'TeamID', 'ScoreDiff': 'AvgTournamentScoreDiff'}, inplace=True)

# Merge extracted features while ensuring no NaN values
tourney_features_w = pd.merge(tourney_stats_w, score_diff_w, on='TeamID', how='left')

# Replace missing values in the score difference with zero to ensure there is no NaN.
tourney_features_w['AvgTournamentScoreDiff'] = tourney_features_w['AvgTournamentScoreDiff'].fillna(0)

# Save feature file
tourney_features_w.to_csv('/kaggle/working/tourney_features_w.csv', index=False)

# Preview final results
print(tourney_features_w[['TeamID', 'TournamentWinRate', 'AvgTournamentScoreDiff']].head())


   TeamID  TournamentWinRate  AvgTournamentScoreDiff
0    3101              0.000                     0.0
1    3103              0.000                     0.0
2    3104              0.500                    15.6
3    3106              0.000                     0.0
4    3107              0.125                     2.0


### Calculating the successful shooting percentage for each team and the extra time rate from the detailed results of the women's tournaments:

In [17]:
# Load detailed tournament results data
tourney_detailed_w = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv')

# Create a copy for the winning teams containing team ID, field goals made (FGM), field goals attempted (FGA), and number of overtime games (NumOT)
winners_w = tourney_detailed_w[['WTeamID', 'WFGM', 'WFGA', 'NumOT']].copy()
winners_w.rename(columns={'WTeamID': 'TeamID', 'WFGM': 'FGM', 'WFGA': 'FGA'}, inplace=True)

# Create a copy for the losing teams containing team ID, points scored (used as FGM approximation), and number of overtime games
losers_w = tourney_detailed_w[['LTeamID', 'LScore', 'NumOT']].copy()
losers_w.rename(columns={'LTeamID': 'TeamID', 'LScore': 'FGM'}, inplace=True)

# Estimate field goal attempts for losing teams using an assumed 45% shooting percentage
losers_w['FGA'] = losers_w['FGM'] / 0.45  # Estimated based on general shooting accuracy

# Merge winners and losers into a single dataset
teams_shots_w = pd.concat([winners_w, losers_w], ignore_index=True)

# Handle missing or invalid values in FGA before calculating FG%
teams_shots_w['FGA'] = teams_shots_w['FGA'].fillna(1)  # Replace NaN with 1 to avoid division errors
teams_shots_w['FGA'] = teams_shots_w['FGA'].clip(lower=1)  # Ensure FGA is at least 1 to prevent division issues

# Compute field goal percentage (FG%)
teams_shots_w['FG%'] = teams_shots_w['FGM'] / teams_shots_w['FGA']

# Fill missing values in NumOT
teams_shots_w['NumOT'] = teams_shots_w['NumOT'].fillna(0)

# Calculate the average field goal percentage per team
fg_percentage_w = teams_shots_w.groupby('TeamID')['FG%'].mean().reset_index().rename(columns={'FG%': 'AvgFieldGoalPercentage'})

# Calculate the average number of overtime games per team
ot_games_w = teams_shots_w.groupby('TeamID')['NumOT'].mean().reset_index().rename(columns={'NumOT': 'AvgOTGames'})

# Merge the extracted features
detailed_features_w = fg_percentage_w.merge(ot_games_w, on='TeamID', how='left')

# Save the features to a CSV file
detailed_features_w.to_csv('/kaggle/working/detailed_features_w.csv', index=False)

# Display the first few rows of the final dataset
print(detailed_features_w.head())


   TeamID  AvgFieldGoalPercentage  AvgOTGames
0    3101                0.450000         0.0
1    3103                0.450000         0.0
2    3104                0.463703         0.0
3    3106                0.450000         0.0
4    3107                0.456250         0.0


### Calculating Win Rate and Average Points Difference in Regular Season Under Pressure for women:

In [18]:
# Load Regular Season Results Data
season_results_w = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')

# Calculate Regular Season Wins and Losses
season_wins_w = season_results_w.groupby('WTeamID').size().reset_index(name='SeasonWins')
season_losses_w = season_results_w.groupby('LTeamID').size().reset_index(name='SeasonLosses')

# Rename columns to unify TeamID
season_wins_w.rename(columns={'WTeamID': 'TeamID'}, inplace=True)
season_losses_w.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Merge wins and losses, replacing NaN with 0
season_stats_w = pd.merge(season_wins_w, season_losses_w, on='TeamID', how='outer').fillna(0)

# Convert columns to integer type after filling NaN values
season_stats_w[['SeasonWins', 'SeasonLosses']] = season_stats_w[['SeasonWins', 'SeasonLosses']].astype(int)

# Calculate Season Win Rate (avoid division by zero)
season_stats_w['SeasonWinRate'] = np.where(
    (season_stats_w['SeasonWins'] + season_stats_w['SeasonLosses']) > 0,
    season_stats_w['SeasonWins'] / (season_stats_w['SeasonWins'] + season_stats_w['SeasonLosses']),
    0
)

# Calculate average points difference for both winning and losing teams
season_results_w['ScoreDiff'] = season_results_w['WScore'] - season_results_w['LScore']
season_results_w['LScoreDiff'] = -season_results_w['ScoreDiff']

# Compute mean score difference for winners and losers
avg_score_diff_win = season_results_w.groupby('WTeamID')['ScoreDiff'].mean().reset_index()
avg_score_diff_loss = season_results_w.groupby('LTeamID')['LScoreDiff'].mean().reset_index()

# Rename columns
avg_score_diff_win.rename(columns={'WTeamID': 'TeamID', 'ScoreDiff': 'AvgSeasonScoreDiff'}, inplace=True)
avg_score_diff_loss.rename(columns={'LTeamID': 'TeamID', 'LScoreDiff': 'AvgSeasonScoreDiff'}, inplace=True)

# Combine winners and losers score differences and take the mean per team
avg_score_diff_w = pd.concat([avg_score_diff_win, avg_score_diff_loss]).groupby('TeamID')['AvgSeasonScoreDiff'].mean().reset_index()

# Merge extracted features
season_features_w = season_stats_w.merge(avg_score_diff_w, on='TeamID', how='left')

# Replace NaN values in score difference with zero
season_features_w['AvgSeasonScoreDiff'] = season_features_w['AvgSeasonScoreDiff'].fillna(0)

# Save feature file
season_features_w.to_csv('/kaggle/working/season_features_w.csv', index=False)

# Data Preview
print(season_features_w.head())

   TeamID  SeasonWins  SeasonLosses  SeasonWinRate  AvgSeasonScoreDiff
0    3101         174           126       0.580000            1.283935
1    3102         165           611       0.212629           -3.935367
2    3103         317           484       0.395755           -1.825544
3    3104         428           397       0.518788            1.137191
4    3105         296           446       0.398922           -2.501129


### Calculating successful shooting percentage and average points conceded for each team from detailed results for the women's regular season:

In [19]:
# Load detailed results for the regular season
season_detailed_w = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv')

# Extract relevant stats for the winning team (field goals made/attempted, points scored, and points allowed)
winners_w = season_detailed_w[['WTeamID', 'WFGM', 'WFGA', 'LScore', 'WScore']].copy()
winners_w.rename(columns={'WTeamID': 'TeamID', 'WFGM': 'FGM', 'WFGA': 'FGA', 
                          'LScore': 'PointsAllowed', 'WScore': 'PointsScored'}, inplace=True)

# Extract relevant stats for the losing team (same stats as winners)
losers_w = season_detailed_w[['LTeamID', 'LScore', 'WFGM', 'WFGA', 'WScore']].copy()
losers_w.rename(columns={'LTeamID': 'TeamID', 'LScore': 'PointsScored', 'WFGM': 'FGM', 
                         'WFGA': 'FGA', 'WScore': 'PointsAllowed'}, inplace=True)

# Combine winners and losers into a single dataset
teams_stats_w = pd.concat([winners_w, losers_w], ignore_index=True)

# Ensure FGA values are non-negative
teams_stats_w['FGA'] = teams_stats_w['FGA'].clip(lower=0)

# Calculate field goal percentage (avoiding division by zero)
teams_stats_w['FG%'] = np.where(teams_stats_w['FGA'] > 0, teams_stats_w['FGM'] / teams_stats_w['FGA'], 0)

# Compute the average field goal percentage for each team
fg_percentage_season_w = teams_stats_w.groupby('TeamID')['FG%'].mean().reset_index().rename(
    columns={'FG%': 'AvgFieldGoalPercentageSeason'}
)

# Compute the average points allowed (defensive strength) for each team
defensive_strength_w = teams_stats_w.groupby('TeamID')['PointsAllowed'].mean().reset_index().rename(
    columns={'PointsAllowed': 'AvgPointsAllowed'}
)

# Compute the average points scored (offensive strength) for each team
offensive_strength_w = teams_stats_w.groupby('TeamID')['PointsScored'].mean().reset_index().rename(
    columns={'PointsScored': 'AvgPointsScored'}
)

# Merge all extracted features into a single dataset
season_detailed_features_w = fg_percentage_season_w.merge(defensive_strength_w, on='TeamID', how='left')
season_detailed_features_w = season_detailed_features_w.merge(offensive_strength_w, on='TeamID', how='left')

# Fill NaN values with 0
season_detailed_features_w = season_detailed_features_w.fillna(0)

# Save the feature dataset
season_detailed_features_w.to_csv('/kaggle/working/season_detailed_features_w.csv', index=False)

# Display the first few rows of the final dataset
print(season_detailed_features_w.head())


   TeamID  AvgFieldGoalPercentageSeason  AvgPointsAllowed  AvgPointsScored
0    3101                      0.461820         66.463333        69.886667
1    3102                      0.451757         68.109170        57.288210
2    3103                      0.438990         68.294505        68.178022
3    3104                      0.445287         64.116910        67.371608
4    3105                      0.428845         65.077098        59.725624


### Calculating win rate and performance difference for each team in a women's secondary game under pressure:

In [20]:
# Load secondary tournament results
file_path = "/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv"
secondary_tourney_results_w = pd.read_csv(file_path)

# Calculate the number of wins and losses in the secondary tournament
secondary_tourney_wins_w = secondary_tourney_results_w.groupby('WTeamID').size().reset_index(name='SecondaryTourneyWins')
secondary_tourney_losses_w = secondary_tourney_results_w.groupby('LTeamID').size().reset_index(name='SecondaryTourneyLosses')

# Rename columns to standardize TeamID across wins and losses
secondary_tourney_wins_w.rename(columns={'WTeamID': 'TeamID'}, inplace=True)
secondary_tourney_losses_w.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Merge win and loss statistics for each team
secondary_tourney_stats_w = pd.merge(secondary_tourney_wins_w, secondary_tourney_losses_w, on='TeamID', how='outer').fillna(0)

# Calculate win rate in the secondary tournament, avoiding division by zero
secondary_tourney_stats_w['SecondaryTourneyWinRate'] = np.where(
    (secondary_tourney_stats_w['SecondaryTourneyWins'] + secondary_tourney_stats_w['SecondaryTourneyLosses']) > 0,
    secondary_tourney_stats_w['SecondaryTourneyWins'] / (secondary_tourney_stats_w['SecondaryTourneyWins'] + secondary_tourney_stats_w['SecondaryTourneyLosses']),
    0
)

# Identify games played under pressure (either in overtime or with a score difference of 5 points or less)
secondary_tourney_results_w['ScoreDiff'] = secondary_tourney_results_w['WScore'] - secondary_tourney_results_w['LScore']
pressure_games_w = secondary_tourney_results_w[
    (secondary_tourney_results_w['NumOT'] > 0) | (secondary_tourney_results_w['ScoreDiff'].abs() <= 5)
]

# Calculate the number of wins and losses in pressure situations
pressure_wins_w = pressure_games_w.groupby('WTeamID').size().reset_index(name='PressureWins')
pressure_losses_w = pressure_games_w.groupby('LTeamID').size().reset_index(name='PressureLosses')

# Rename columns to standardize TeamID across wins and losses in pressure games
pressure_wins_w.rename(columns={'WTeamID': 'TeamID'}, inplace=True)
pressure_losses_w.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Merge pressure game statistics
pressure_stats_w = pd.merge(pressure_wins_w, pressure_losses_w, on='TeamID', how='outer').fillna(0)

# Calculate win rate under pressure, avoiding division by zero
pressure_stats_w['SecondaryTourneyPressureWinRate'] = np.where(
    (pressure_stats_w['PressureWins'] + pressure_stats_w['PressureLosses']) > 0,
    pressure_stats_w['PressureWins'] / (pressure_stats_w['PressureWins'] + pressure_stats_w['PressureLosses']),
    0
)

# Merge secondary tournament statistics with pressure game statistics
secondary_tourney_features_w = pd.merge(
    secondary_tourney_stats_w[['TeamID', 'SecondaryTourneyWinRate']],
    pressure_stats_w[['TeamID', 'SecondaryTourneyPressureWinRate']],
    on='TeamID',
    how='left'
).fillna(0)

# Save the final dataset with extracted features
secondary_tourney_features_w.to_csv('/kaggle/working/secondary_tourney_features_w.csv', index=False)

# Display the first few rows of the final dataset
print(secondary_tourney_features_w.head())


   TeamID  SecondaryTourneyWinRate  SecondaryTourneyPressureWinRate
0    3101                 0.166667                         0.000000
1    3102                 0.500000                         1.000000
2    3103                 0.000000                         0.000000
3    3104                 0.692308                         0.666667
4    3107                 0.000000                         0.000000


### Merge all extracted feature files for women:

In [21]:
# Load extracted feature files efficiently
def load_feature_file(filename, columns):
    return pd.read_csv(filename, usecols=columns)

# Define required columns for each file
feature_files = {
    'conference_summary_w.csv': ['TeamID', 'TeamConferenceWinRate'],
    'tourney_features_w.csv': ['TeamID', 'TournamentWinRate', 'AvgTournamentScoreDiff'],
    'detailed_features_w.csv': ['TeamID', 'AvgFieldGoalPercentage', 'AvgOTGames'],
    'season_features_w.csv': ['TeamID', 'SeasonWinRate', 'AvgSeasonScoreDiff'],
    'season_detailed_features_w.csv': ['TeamID', 'AvgFieldGoalPercentageSeason', 'AvgPointsAllowed', 'AvgPointsScored'],
    'secondary_tourney_features_w.csv': ['TeamID', 'SecondaryTourneyWinRate', 'SecondaryTourneyPressureWinRate']
}

# Load and merge feature data
merged_df_w = None
for file, cols in feature_files.items():
    df = load_feature_file(file, cols)
    merged_df_w = df if merged_df_w is None else merged_df_w.merge(df, on='TeamID', how='left')

# Check the number of unique teams in each file
for file_name, cols in feature_files.items():
    df = load_feature_file(file_name, cols)
    print(f"{file_name}: {df['TeamID'].nunique()} unique teams")

# Print missing values before filling
print("Missing values before filling:", merged_df_w.isna().sum())

# Fill missing values with column mean, then replace any remaining NaN with 0
merged_df_w.fillna(merged_df_w.mean(numeric_only=True), inplace=True)
merged_df_w.fillna(0, inplace=True)

# Print missing values after filling
print("Missing values after filling:", merged_df_w.isna().sum())

# Save the merged file
merged_df_w.to_csv('/kaggle/working/merged_team_features_w.csv', index=False)

# Show preliminary results
print(merged_df_w.head())


conference_summary_w.csv: 361 unique teams
tourney_features_w.csv: 279 unique teams
detailed_features_w.csv: 225 unique teams
season_features_w.csv: 369 unique teams
season_detailed_features_w.csv: 366 unique teams
secondary_tourney_features_w.csv: 285 unique teams
Missing values before filling: TeamID                               0
TeamConferenceWinRate                0
TournamentWinRate                   83
AvgTournamentScoreDiff              83
AvgFieldGoalPercentage             136
AvgOTGames                         136
SeasonWinRate                        0
AvgSeasonScoreDiff                   0
AvgFieldGoalPercentageSeason         1
AvgPointsAllowed                     1
AvgPointsScored                      1
SecondaryTourneyWinRate             76
SecondaryTourneyPressureWinRate     76
dtype: int64
Missing values after filling: TeamID                             0
TeamConferenceWinRate              0
TournamentWinRate                  0
AvgTournamentScoreDiff             0
AvgFi

### Create all possible combinations of teams so that each pair consists of two different women's teams:

In [22]:
from itertools import combinations

# Load merged features
file_path = "/kaggle/working/merged_team_features_w.csv"
df = pd.read_csv(file_path)

# Extract all possible team pairs efficiently
team_pairs_w = combinations(df['TeamID'], 2)

# Create a DataFrame to store the differences
columns_w = ['TeamID1', 'TeamID2'] + [f'{col}_diff' for col in df.columns if col != 'TeamID'] + ['WinProbability']
pairs_df_w = []

# Convert df to a dictionary with TeamID as index for faster lookups
df_dict = df.set_index('TeamID').to_dict(orient='index')

# Process team pairs
for team1, team2 in team_pairs_w:
    team1_data_w = df_dict[team1]
    team2_data_w = df_dict[team2]

    row = {
        'TeamID1': team1,
        'TeamID2': team2,
    }

    feature_diffs_w = []
    for col in df.columns:
        if col != 'TeamID':
            diff = team1_data_w[col] - team2_data_w[col]
            row[f'{col}_diff'] = diff
            feature_diffs_w.append(diff)

    # Compute win probability using sigmoid function
    row['WinProbability'] = round(1 / (1 + np.exp(-sum(feature_diffs_w))), 2)

    pairs_df_w.append(row)

# Convert to DataFrame
pairs_df_w = pd.DataFrame(pairs_df_w, columns=columns_w)

# Save to CSV
pairs_df_w.to_csv("/kaggle/working/team_pairs_diff_w.csv", index=False)

print("The file team_pairs_diff_w.csv was created successfully!")


The file team_pairs_diff_w.csv was created successfully!


In [23]:
team_pairs_diff_w = pd.read_csv("/kaggle/working/team_pairs_diff_w.csv")
team_pairs_diff_w

,TeamID1,TeamID2,TeamConferenceWinRate_diff,TournamentWinRate_diff,AvgTournamentScoreDiff_diff,AvgFieldGoalPercentage_diff,AvgOTGames_diff,SeasonWinRate_diff,AvgSeasonScoreDiff_diff,AvgFieldGoalPercentageSeason_diff,AvgPointsAllowed_diff,AvgPointsScored_diff,SecondaryTourneyWinRate_diff,SecondaryTourneyPressureWinRate_diff,WinProbability
0,3101,3102,0.293103,-0.208687,-5.931126,-0.000167,-0.02309,0.367371,5.219302,0.010063,-1.645837,12.598457,-0.333333,-1.000000,1.00
1,3101,3103,0.133333,0.000000,0.000000,0.000000,0.00000,0.184245,3.109479,0.022830,-1.831172,1.708645,0.166667,0.000000,0.97
2,3101,3104,0.241935,-0.500000,-15.600000,-0.013703,0.00000,0.061212,0.146744,0.016533,2.346423,2.515059,-0.525641,-0.666667,0.00
3,3101,3105,0.333333,-0.208687,-5.931126,-0.000167,-0.02309,0.181078,3.785064,0.032975,1.386236,10.161043,-0.209459,-0.301579,1.00
4,3101,3106,-0.050000,0.000000,0.000000,0.000000,0.00000,0.133385,5.121274,0.040375,0.384148,11.990739,-0.209459,-0.301579,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64975,3475,3477,0.333333,0.000000,0.000000,0.000000,0.00000,0.228298,3.629273,0.008936,-7.035372,2.843837,0.123874,0.198421,0.58
64976,3475,3478,0.333333,0.000000,0.000000,0.000000,0.00000,0.196886,6.466232,0.010726,-0.058608,12.825549,0.500000,0.500000,1.00
64977,3476,3477,-0.666667,0.000000,0.000000,0.000000,0.00000,-0.077495,0.307895,0.009789,-4.616094,-7.112004,0.000000,0.000000,0.00
64978,3476,3478,-0.666667,0.000000,0.000000,0.000000,0.00000,-0.108907,3.144854,0.011579,2.360670,2.869709,0.376126,0.301579,1.00


### Model for predicting the probability of the team with the smallest ID for women winning:

### Using XGBoost:

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import optuna

# Define feature columns (all '_diff' columns) and target variable ('WinProbability')
feature_columns = [col for col in pairs_df_w.columns if col.endswith('_diff')]
XW = pairs_df_w[feature_columns]
yw = pairs_df_w['WinProbability']

# Split the dataset into training (80%) and testing (20%) ensuring reproducibility
XW_train, XW_test, yw_train, yw_test = train_test_split(XW, yw, test_size=0.2, random_state=42)

# Handle missing values using SimpleImputer (replace NaNs with the median)
imputer = SimpleImputer(strategy='median')
XW_train = imputer.fit_transform(XW_train)
XW_test = imputer.transform(XW_test)

print(f"Training set size: {XW_train.shape}, Testing set size: {XW_test.shape}")

# Define the Optuna optimization function
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.15),  # Reduced for stability
        'max_depth': trial.suggest_int('max_depth', 5, 6),  # Reduced complexity
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # Increased for better generalization
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),  # Increased for better generalization
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 7.0, 15.0),  # Increased L2 regularization to reduce overfitting
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # Added to prevent overfitting
        'random_state': 42
    }
    
    # Train the model with early stopping to prevent overfitting
    model = XGBRegressor(**params)
    model.fit(XW_train, yw_train, eval_set=[(XW_test, yw_test)], early_stopping_rounds=30, verbose=False)
    
    # Make predictions and clip values between 0 and 1
    yw_pred = np.clip(model.predict(XW_test), 0, 1)
    
    # Return Mean Squared Error (MSE) for evaluation
    return mean_squared_error(yw_test, yw_pred)

# Run Optuna to optimize hyperparameters
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=75)  # Increased trials for better optimization

# Retrieve the best hyperparameters found by Optuna
best_params = study.best_params
print("Best Hyperparameters: ", best_params)

# Train the final model with optimized hyperparameters and early stopping
best_model = XGBRegressor(**best_params)
best_model.fit(XW_train, yw_train, eval_set=[(XW_test, yw_test)], early_stopping_rounds=30, verbose=False)

# Generate final predictions and ensure values are between 0 and 1
yw_pred_full = np.clip(best_model.predict(XW), 0, 1)
pairs_df_w['WinProbability'] = yw_pred_full

# Format output to match the required submission format
season = 2025  # Adjust the season as needed
pairs_df_w['ID'] = pairs_df_w.apply(lambda row: f"{season}_{int(row['TeamID1'])}_{int(row['TeamID2'])}", axis=1)
pairs_df_w['Pred'] = pairs_df_w['WinProbability'].round(1)  # Round probabilities to one decimal place
submission_df_w = pairs_df_w[['ID', 'Pred']]

# Save the submission file
submission_df_w.to_csv("/kaggle/working/submission_w.csv", index=False)
print("The file 'submission_w.csv' was created successfully!")

# Calculate and display MSE on both training and test sets
yw_pred_train = np.clip(best_model.predict(XW_train), 0, 1)
yw_pred_test = np.clip(best_model.predict(XW_test), 0, 1)
train_mse = mean_squared_error(yw_train, yw_pred_train)
test_mse = mean_squared_error(yw_test, yw_pred_test)
print(f"Mean Squared Error on Training Set: {train_mse:.4f}")
print(f"Mean Squared Error on Test Set: {test_mse:.4f}")


[I 2025-03-03 08:46:26,723] A new study created in memory with name: no-name-f6f063ee-aa7b-482e-96ac-b4a803314def


Training set size: (51984, 12), Testing set size: (12996, 12)


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2025-03-03 08:46:28,552] Trial 0 finished with value: 0.0045203912030867075 and parameters: {'n_estimators': 287, 'learning_rate': 0.1450714306409916, 'max_depth': 6, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'reg_alpha': 1.5599452033620265, 'reg_lambda': 7.464668897345596, 'min_child_weight': 9}. Best is trial 0 with value: 0.0045203912030867075.
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2025-03-03 08:46:30,558] Trial 1 finished with value: 0.004801478086382166 and parameters: {'n_

Best Hyperparameters:  {'n_estimators': 589, 'learning_rate': 0.12005930969503895, 'max_depth': 6, 'subsample': 0.8556816841827655, 'colsample_bytree': 0.958148829739241, 'reg_alpha': 0.9929076578602127, 'reg_lambda': 12.249865929878183, 'min_child_weight': 4}
The file 'submission_w.csv' was created successfully!
Mean Squared Error on Training Set: 0.0014
Mean Squared Error on Test Set: 0.0032


In [25]:
submission_w = pd.read_csv("/kaggle/working/submission_w.csv") 
submission_w

,ID,Pred
0,2025_3101_3102,1.0
1,2025_3101_3103,0.9
2,2025_3101_3104,0.0
3,2025_3101_3105,1.0
4,2025_3101_3106,1.0
...,...,...
64975,2025_3475_3477,0.4
64976,2025_3475_3478,1.0
64977,2025_3476_3477,0.0
64978,2025_3476_3478,1.0


### Ensure that there are no empty values in the prediction model:

In [26]:
# Download the file
submission_w = pd.read_csv('submission_w.csv')

# Check for blank values ​​in ID column
missing_id = submission_w['ID'].isna().sum()
print(f"Number of empty values ​​in ID: {missing_id}")

# Extract TeamID1 and TeamID2 from ID
submission_w[['Season', 'TeamID1', 'TeamID2']] = submission_w['ID'].str.split('_', expand=True)

# Check for empty values ​​in TeamID1 and TeamID2
missing_teamid1 = submission_w['TeamID1'].isna().sum()
missing_teamid2 = submission_w['TeamID2'].isna().sum()

print(f"Number of empty values ​​in TeamID1: {missing_teamid1}")
print(f"Number of empty values ​​in TeamID2: {missing_teamid2}")

# If there are no empty values
if missing_id == 0 and missing_teamid1 == 0 and missing_teamid2 == 0:
    print("\nThere are no empty values ​​in ID, TeamID1, or TeamID2.")
else:
    print("\nThere are blank values ​​in some columns, check the data.")

Number of empty values ​​in ID: 0
Number of empty values ​​in TeamID1: 0
Number of empty values ​​in TeamID2: 0

There are no empty values ​​in ID, TeamID1, or TeamID2.


## Stage 3: Merge the prediction models for men's and women's teams into one prediction model:

In [27]:
# Load the reference submission file (to maintain the original order)
submission_stage2_path = "/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage2.csv"
submission_m_path = "submission_m.csv"
submission_w_path = "submission_w.csv"

submission_stage2 = pd.read_csv(submission_stage2_path)
submission_m = pd.read_csv(submission_m_path)
submission_w = pd.read_csv(submission_w_path)

# Convert predictions into dictionaries for fast lookup
predictions_m = dict(zip(submission_m["ID"], submission_m["Pred"]))
predictions_w = dict(zip(submission_w["ID"], submission_w["Pred"]))

# Update predictions in the original file, keeping the same order
submission_stage2["Pred"] = submission_stage2["ID"].map(lambda x: predictions_m.get(x, predictions_w.get(x, 0.5)))

# Keep only the required columns: 'ID' and 'Pred'
submission_stage2 = submission_stage2[['ID', 'Pred']]

# Save the final merged submission file
submission = "/kaggle/working/submission.csv"
submission_stage2.to_csv(submission, index=False)

print(f"Submission file saved at: {submission}")


Submission file saved at: /kaggle/working/submission.csv


## Ensure that there are no empty values ​​in the built-in prediction model:

In [28]:
# Download the resulting file
submission = pd.read_csv('/kaggle/working/submission.csv')

# Ensure that the ID column exists.
if 'ID' in submission.columns:
    # Split ID column to extract TeamID1 and TeamID2
    submission[['Season', 'TeamID1', 'TeamID2']] = submission['ID'].str.split('_', expand=True)

    # Convert TeamID1 and TeamID2 to integers
    submission['TeamID1'] = pd.to_numeric(submission['TeamID1'], errors='coerce')
    submission['TeamID2'] = pd.to_numeric(submission['TeamID2'], errors='coerce')

    # Find any empty values.
    missing_values = submission[['TeamID1', 'TeamID2']].isna().sum()

    print("Empty values ​​in columns:")
    print(missing_values)

    # Check if there are any rows with empty values.
    if missing_values.sum() == 0:
        print("\nThere are no empty values ​​in TeamID1 or TeamID2.")
    else:
        print("\nThere are empty values ​​in TeamID1 or TeamID2, please check the data.")

else:
    print("Column 'ID' is missing from the file.")

Empty values ​​in columns:
TeamID1    0
TeamID2    0
dtype: int64

There are no empty values ​​in TeamID1 or TeamID2.


In [29]:
# Read the saved submission file
submission = pd.read_csv("/kaggle/working/submission.csv")

# Check the info of the submission DataFrame
submission.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131407 entries, 0 to 131406
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      131407 non-null  object 
 1   Pred    131407 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.0+ MB


In [30]:
SampleSubmissionStage2 = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage2.csv")
SampleSubmissionStage2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131407 entries, 0 to 131406
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      131407 non-null  object 
 1   Pred    131407 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.0+ MB


In [31]:
submission

,ID,Pred
0,2025_1101_1102,0.0
1,2025_1101_1103,0.0
2,2025_1101_1104,0.0
3,2025_1101_1105,0.1
4,2025_1101_1106,0.0
...,...,...
131402,2025_3477_3479,0.5
131403,2025_3477_3480,0.5
131404,2025_3478_3479,0.5
131405,2025_3478_3480,0.5
